In [6]:
# Imports
import mpmath
import numpy as np
import sympy as sp
from sympy import nsolve
from sympy.solvers import solve

# Inputs
# Geometric properties (unit)
R = 6  # (mm)
W = 72  # (mm)   # R/W must be < 1/2

# Material properties (unit)
XTL = 845.1  # (MPa) unnotched strength of the laminate
KIC = 1517  # (MPamm^0.5) mode I fracture toughness of the laminate

E1 = 171400  # (MPa)
E2 = 9100  # (MPa)
G12 = 5300  # (MPa)
v12 = 0.3
v21 = E2*(v12/E1)

# Ply parameters
number_of_plies = 24
ply_thickness = [0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002]  # (m) Enter value for each ply
ply_angle = [90,0,45,-45,-45,45,0,90,90,0,45,-45,-45,45,0,90,90,0,45,-45,-45,45,0,90] 
            # (deg) Enter value for each ply

if len(ply_angle) != number_of_plies:
    print("Error. Please enter angle of every ply")
    exit()

if len(ply_thickness) != number_of_plies:
    print("Error. Please enter thickness of every ply")
    exit()

# Calculate Aij components of the laminate in-plane stiffness
# Calculating list of trigonometric values, where m = cosθ and n = sinθ
m_values = []
n_values = []
for i in range(number_of_plies):
    m = np.cos(np.radians(ply_angle[i]))
    m_values.append(m)
    n = np.sin(np.radians(ply_angle[i]))
    n_values.append(n)

# Calculating Q values (constant for every ply)
Q11 = E1/(1-(v12*v21))
Q12 = (v21*E1)/(1-(v12*v21))
Q21 = (v12*E2)/(1-(v12*v21))
Q22 = E2/(1-(v12*v21))
Q66 = G12

# Calculating Qbar values (different for every ply: depends on ply orientation)
Qbar11_values = []
Qbar12_values = []
Qbar21_values = []
Qbar22_values = []
Qbar66_values = []

for i in range(number_of_plies):
    Qbar11 = ((m_values[i]**4)*Q11)+((n_values[i]**4)*Q22)+((2*(m_values[i]**2)*(n_values[i]**2))*Q12)+((4*(m_values[i]**2)*(n_values[i]**2))*Q66)
    Qbar11_values.append(Qbar11)
    Qbar12 = (((m_values[i]**2)*(n_values[i]**2))*Q11)+(((m_values[i]**2)*(n_values[i]**2))*Q22)+((m_values[i]**4)*Q12)+((-4*(m_values[i]**2)*(n_values[i]**2))*Q66)
    Qbar12_values.append(Qbar12)
    Qbar21 = (((m_values[i]**2)*(n_values[i]**2))*Q11)+(((m_values[i]**2)*(n_values[i]**2))*Q22)+(((m_values[i]**4)+(n_values[i]**4))*Q21)+((-4*(m_values[i]**2)*(n_values[i]**2))*Q12)
    Qbar21_values.append(Qbar21)
    Qbar22 = ((n_values[i]**4)*Q11)+((m_values[i]**4)*Q22)+((2*(m_values[i]**2)*(n_values[i]**2))*Q12)+((4*(m_values[i]**2)*(n_values[i]**2))*Q66)
    Qbar22_values.append(Qbar22)  
    Qbar66 = (((m_values[i]**2)*(n_values[i]**2))*Q11)+(((m_values[i]**2)*(n_values[i]**2))*Q22)+((-2*(m_values[i]**2)*(n_values[i]**2))*Q12)+((((m_values[i]**2)-(n_values[i]**2))**2)*(Q66))
    Qbar66_values.append(Qbar66)

# Calculating A values (components of the laminate in-plane stiffness matrix)
A11_values = []
A12_values = []
A21_values = []
A22_values = []
A66_values = []

for i in range(number_of_plies):
    A11 = Qbar11_values[i]*ply_thickness[i]
    A11_values.append(A11)
    A12 = Qbar12_values[i]*ply_thickness[i]
    A12_values.append(A12)
    A21 = Qbar21_values[i]*ply_thickness[i]
    A21_values.append(A21)
    A22 = Qbar22_values[i]*ply_thickness[i]
    A22_values.append(A22)
    A66 = Qbar11_values[i]*ply_thickness[i]
    A66_values.append(A66)

A11 = sum(A11_values)
A12 = sum(A12_values)
A21 = sum(A21_values)
A22 = sum(A22_values)
A66 = sum(A66_values)

# Calculate M (geometric parameter)
M = np.sqrt((np.sqrt(1-8*((3*(1-2*(R/W)))/(2+(1-2*(R/W))**3)-1))-1)/(2*((2*(R/W))**2)))

# Calculating KT∞
KT_inf = 1 + np.sqrt((2/A22)*((np.sqrt(A11*A22))-A12+((A11*A22-(A12**2))/(2*A66))))

# Calculating RK
RK = ((3*(1-2*(R/W)))/(2+(1-2*(R/W))**3)+(0.5*((2*R*M)/W)**6)*(KT_inf-3)*(1-((2*R*M)/W)**2))**-1

# Calculating fn
a = sp.Symbol('a')
fn = 1 + 0.358*(R/a) + 1.425*(R/a)**2 - 1.578*(R/a)**3 + 2.156*(R/a)**4

# Calculating Fh
Fh = sp.sqrt(1-(R/a))*fn

# Calculating Fw
Fw = sp.sqrt(sp.sec(mpmath.radians((np.pi*R)/W)) * sp.sec(mpmath.radians((np.pi*a)/W)))

# Solving for l
# Integrating denominator
x = sp.Symbol('x')
l = sp.Symbol('l')
integral_d = sp.integrate((2 + (R/x)**2 + 3*(R/x)**4 - (KT_inf-3)*((5*(R/x)**6)-7*(R/x)**8)),(x,R,R+l))

# Calculating denominator
denom = RK**2*integral_d**2

# Integrating numerator using the Simpson's Rule
ul = R+l  # upper limit
ll = R  # lower limit
n = 20  # number of strips (must be even)
h = (ul-ll)/n  # height of each strip
f = ((Fh*Fw)**2)*a

a_values = []
for i in range(n+1):
    dh = ll + h*i
    a_values.append(dh)

a_values_odd = a_values[1::2]  # creates list of only values in odd index positions
a_values_even = a_values[2:n:2]  # creates list of only values in even index positions 
                                 # excluding first and last

integral_n_odd_values = []
for i in range(len(a_values_odd)):
    integral_n_odd = 4*(f.subs(a,a_values_odd[i]))
    integral_n_odd_values.append(integral_n_odd)

integral_n_even_values = []
for i in range(len(a_values_even)):
    integral_n_even = 2*(f.subs(a,a_values_even[i]))
    integral_n_even_values.append(integral_n_even)

integral_n = (h/3)*(f.subs(a,a_values[0])+(f.subs(a,a_values[n]))+sum(integral_n_odd_values)+sum(integral_n_even_values))

# Calculating numerator
numer = 4*l*np.pi*integral_n

# Calculating l
l_equation = (numer/denom)-(KIC/XTL)**2
l = nsolve(l_equation, l, (0,W-R))

# Calculating stress distribution across x-axis
σ_inf = sp.Symbol('σ_inf')

# Integrating and calculating LHS of equation 1 to solve for σ_inf
ul = R+l  # upper limit
ll = R  # lower limit
n = 10  # number of strips (must be even)
h = (ul-ll)/n # height of each strip
f = ((RK*σ_inf)/2)*((2+(R/x)**2)+3*((R/x)**4)-((KT_inf-3)*((5*(R/x)**6)-(7*(R/x)**8))))

x_values = []
for i in range(n+1):
    dh = ll + h*i
    x_values.append(dh)

x_values_odd = x_values[1::2]  # creates list of only values in odd index positions
x_values_even = x_values[2:n:2]  # creates list of only values in even index positions 
                                 # excluding first and last

integral_n_odd_values = []
for i in range(len(x_values_odd)):
    integral_n_odd = 4*(f.subs(x,x_values_odd[i]))
    integral_n_odd_values.append(integral_n_odd)

integral_n_even_values = []
for i in range(len(x_values_even)):
    integral_n_even = 2*(f.subs(x,x_values_even[i]))
    integral_n_even_values.append(integral_n_even)

integral_1 = (h/3)*(f.subs(x,x_values[0])+(f.subs(x,x_values[n]))+sum(integral_n_odd_values)+sum(integral_n_even_values))

eq_1 = ((1/l)*integral_1)-XTL
σ_inf = solve(eq_1,σ_inf)

# Printing outputs
print("Crack length at failure = " + "[" + str(l) + "]" + " mm")
print("Remote stress at failure = " + str(σ_inf) + " MPa")

Crack length at failure = [1.74959844921494] mm
Remote stress at failure = [369.863142547110] MPa
